In [1]:
!pip install langchain langchain-core langchain-community
!pip install openai
!pip install pypdf
!pip install tiktoken
!pip install chromadb

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf')
pages = loader.load_and_split()

In [3]:
import tiktoken

def tiktoken_len(text):
  tokens = tiktoken.encoding_for_model('gpt-4o-mini')
  return len(tokens.encode(text))

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50,
    length_function = tiktoken_len,
)

In [5]:
docs = text_splitter.split_documents(pages)
docs[0]

Document(metadata={'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf', 'page': 0, 'page_label': '1'}, page_content='2021.05이노비즈 정책브리프I n n o b i z  P o l i c y  B r i e f\n탄소중립 대응을 위한정부 정책과 동향\n탄소중립 대응을 위한 정부 정책과 동향–2021-04호–발행처이노비즈정책연구원–발행인임병훈–편집인김세종–http://www.innobiz.or.kr')

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['OPENAI_API_VERSION'] = '2023-05-15'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://labuser24-aiservice-002.openai.azure.com'

In [7]:
from langchain.embeddings import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    azure_deployment='dev-text-embedding-ada-002',
    chunk_size = 500,
)

<ipython-input-7-56a9d52e716c>:3: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(


In [8]:
from langchain.vectorstores import Chroma

db = Chroma.from_documents(docs, embedding_model)

In [10]:
query = '장기저탄소발전전략'
db.similarity_search(query)

[Document(metadata={'page': 5, 'page_label': '6', 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='02탄소중립 대응을 위한 정부 정책1)탄소중립 2050 추진전략∙우리나라 정부는 지난 2020년 12월 7일 관계부처 합동으로 ‘2050 탄소중립 추진전략’을 발표함- 정부는 탄소중립이 어렵지만 가야할 길이라고 인식하는 한편 산업구조 저탄소화 및 신산업 육성 등 선도적 대응이 필요할 것이라고 말함 - 정부는 탄소중립을 실현하는 가운데 지속가능한 경제성장과 삶의 질 향상을 동시 달성하기 위해 △경제구조 저탄소화, △저탄소 산업생태계 조성, △탄소중립사회로의 공정전환의 3대 정책방향과 △탄소중립 제도기반 강화라는 3+1의 전략을 추진하고자 함 - 또한 3대 정책방향에 따른 10대 과제를 선정하여 신경제·사회구조 시스템 구축을 실현하겠다는 의지를 표명함 ① 경제구조의 저탄소화- (에너지 전환 가속화) 탄소중립을 위해 화석연료에 대한 높은 의존도를 줄이고, 재생 에너지로의 전환 가속화 추진- 산업 분야에서는 재생에너지, 수소, 에너지 IT 등 3대 에너지 신산업 육성에 집중- (고탄소 산업구조 혁신) 철강, 석유화학 등 탄소 다배출 업종에 대한 기술개발 지원, 고탄소 중소기업 대상 맞춤형 공정개선 지원 등을 통해 저탄소 산업구조로의 전환 추진- 금속･화학제품제조업 등 다배출 중소기업을 대상으로 공정진단을 통해 맞춤형 공정개선‧설비보급을 지원하는 등 산업 전과정(연‧원료→공정→제품→소비‧자원 순환)에서의 저탄소 전환을 촉진- (미래모빌리티로 전환) 친환경차 가격·충전·수요 혁신을 통해 수소·전기차 생산, 보급 확대, 전국 2천만 세대 전기차'),
 Document(metadata={'page': 5, 'page_label': '6', 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf

In [14]:
results = db.similarity_search_with_relevance_scores(query, k=3)

In [15]:
results[0][0].page_content

'02탄소중립 대응을 위한 정부 정책1)탄소중립 2050 추진전략∙우리나라 정부는 지난 2020년 12월 7일 관계부처 합동으로 ‘2050 탄소중립 추진전략’을 발표함- 정부는 탄소중립이 어렵지만 가야할 길이라고 인식하는 한편 산업구조 저탄소화 및 신산업 육성 등 선도적 대응이 필요할 것이라고 말함 - 정부는 탄소중립을 실현하는 가운데 지속가능한 경제성장과 삶의 질 향상을 동시 달성하기 위해 △경제구조 저탄소화, △저탄소 산업생태계 조성, △탄소중립사회로의 공정전환의 3대 정책방향과 △탄소중립 제도기반 강화라는 3+1의 전략을 추진하고자 함 - 또한 3대 정책방향에 따른 10대 과제를 선정하여 신경제·사회구조 시스템 구축을 실현하겠다는 의지를 표명함 ① 경제구조의 저탄소화- (에너지 전환 가속화) 탄소중립을 위해 화석연료에 대한 높은 의존도를 줄이고, 재생 에너지로의 전환 가속화 추진- 산업 분야에서는 재생에너지, 수소, 에너지 IT 등 3대 에너지 신산업 육성에 집중- (고탄소 산업구조 혁신) 철강, 석유화학 등 탄소 다배출 업종에 대한 기술개발 지원, 고탄소 중소기업 대상 맞춤형 공정개선 지원 등을 통해 저탄소 산업구조로의 전환 추진- 금속･화학제품제조업 등 다배출 중소기업을 대상으로 공정진단을 통해 맞춤형 공정개선‧설비보급을 지원하는 등 산업 전과정(연‧원료→공정→제품→소비‧자원 순환)에서의 저탄소 전환을 촉진- (미래모빌리티로 전환) 친환경차 가격·충전·수요 혁신을 통해 수소·전기차 생산, 보급 확대, 전국 2천만 세대 전기차'

In [19]:
db2 = Chroma.from_documents(docs, embedding_model, persist_directory='./db')

In [20]:
db3 = Chroma(persist_directory='./db', embedding_function=embedding_model)

In [21]:
db3.similarity_search('녹색성장')

[Document(metadata={'page': 8, 'page_label': '9', 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='06\n이노비즈협회'),
 Document(metadata={'page': 10, 'page_label': '11', 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='08\n이노비즈협회'),
 Document(metadata={'page': 7, 'page_label': '8', 'source': '/content/[정책브리프 2021-04] 탄소중립 대응을 위한 정부 정책과 동향.pdf'}, page_content='이노비즈 정책브리프장기저탄소발전전략(LEDS)∙우리나라는 녹색성장위원회 심의와 국무회의 의결을 거쳐 지난해 말 LEDS를 UN에 제출하였으며 부문별 전략은 다음과 같음- (에너지공급) 화석연료 발전 중심의 전력공급 체계를 재생에너지와 그린수소 중심으로 전환하고 이산화탄소포집 기술 등을 적극적으로 활용하여 전력부문의 탄소중립을 달성- (산업) 미래 신기술, 에너지효율 향상, 순환경제 실현으로 지속가능한 탄소중립 산업 생태계를 구축- (수송) 청정 에너지원(전기·수소)을 동력으로 하는 수송수단(자동차, 철도, 항공기, 선박)을 확대하고 디지털 기술을 활용한 자율주행차, 교통 수요관리를 통해 수송부문 탄소중립 기반을 조성- (건물) 단열과 기밀성능을 강화하고 에너지고효율 제품 사용을 확대하여 건물에서 사용되는 에너지를 최소화하고 태양광, 지열 등 건물 내 재생에너지 보급을 촉진하여 건물 에너지 자급자족을 실현- (농축수산) 농축수산의 스마트화 촉진과 청정에너지 사용 확대를 통해 친환경 농축수산업 체계를 구축- (탄소흡수원) 산림, 갯벌, 습지 등 자연·생태 기반 솔루션 강화로 탄소흡수 능력을 높여 우리나라 탄소중립 달성에 기여- (이